In [1]:
import torch

import sys
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/") # PATH TO PYSEQM

import seqm
from seqm.seqm_functions.constants import Constants
from seqm.seqm_functions.make_dm_guess import make_dm_guess
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from seqm.seqm_functions.read_xyz import read_xyz

from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import os
import numpy as np

seqm.seqm_functions.scf_loop.debug=True
seqm.seqm_functions.scf_loop.MAX_ITER=500

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Decorating your function! <function KSA_XL_BOMD.one_step at 0x7fc71f3b7550>


In [2]:
species = torch.as_tensor([[16,16],[22,22],[22,16],[35,17],[24,22]],dtype=torch.int64, device=device)

coordinates = torch.tensor([
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
                 ], device=device)


# species = torch.as_tensor([[17,17,17,17,6]],dtype=torch.int64, device=device)

# coordinates = torch.tensor([
#                   [
#                    [0.0000,    0.0000,    1.7700],
#                    [1.7700,    0.0000,    0.0000],
#                    [-1.250,    1.2500,    0.0000],
#                    [-1.250,   -1.2500,    0.0000],
#                    [0.0000,    0.0000,    0.0000]
#                   ]
#                  ], device=device)

const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'PM6',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-5,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0,0.2], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : False,
                    "Hf_flag": True,
                    'scf_backward' : 0,
                    'UHF' : False,
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)
esdriver = Electronic_Structure(seqm_parameters).to(device)
esdriver(molecules)

/home/maxim/Projects/git2/PYSEQM_dev/seqm/seqm_functions/diag_d.py:109: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2794.)
  e0,v = torch.symeig(x0,eigenvectors=True,upper=True)


scf  0 81.77464670270234 5 0.7664718627929688
scf  1 10.252213780863599 5 0.08469486236572266
scf  2 23.774032921933326 5 0.12021541595458984
scf  3 12.838189019431297 5 0.1258869171142578
scf  4 16.877663926916853 5 0.12302637100219727
scf  5 7.143621359420251 5 0.08651280403137207
scf  6 11.713543523270971 5 0.0847170352935791
scf  7 1.2483319276588531 5 0.11952424049377441
scf  8 7.380488917169146 5 0.12572169303894043
scf  9 1.9144973080698833 5 0.12337398529052734
scf  10 4.0471069774570765 4 0.08586764335632324
scf  11 2.0310655341724555 4 0.07078075408935547
scf  12 1.7851383168917891 4 0.08732485771179199
scf  13 0.9660046386065346 4 0.10678863525390625
scf  14 0.5795287062667853 4 0.10695052146911621
scf  15 0.48607489752069455 4 0.10007524490356445
scf  16 0.6604478141878474 4 0.0711374282836914
scf  17 0.7709383145672746 4 0.07080674171447754
scf  18 0.7375803411068489 4 0.08192014694213867
scf  19 0.58379801152509 4 0.10560154914855957
scf  20 0.39829958300521184 4 0.108685

# Total E

In [4]:
molecules.Etot[0] # S2. MOPAC Etot = -333.8287 EV

tensor(-333.8298, device='cuda:0')

In [6]:
molecules.Etot[1] # Ti2. MOPAC Etot = -120.0651 EV

tensor(-120.0663, device='cuda:0')

In [7]:
molecules.Etot[2] # TiS. MOPAC Etot = -221.2701 EV  !!!!!!!!!!!!!!!!!!!!!

tensor(-221.2703, device='cuda:0')

In [8]:
molecules.Etot[3] # BrCl. MOPAC Etot = -458.5855 EV

tensor(-458.5839, device='cuda:0')

In [10]:
molecules.Etot[4] # CrTi. MOPAC Etot = -150.3163 EV  !!!!!!!!!!!!!!!!!!!!!

tensor(-255.8132, device='cuda:0')

# Nuclear repulsion

In [11]:
molecules.Enuc[0] # S2. MOPAC Enuc = 290.8693 EV

tensor(290.8621, device='cuda:0')

In [12]:
molecules.Enuc[1] # Ti2. MOPAC Enuc = 97.3923 EV

tensor(97.3923, device='cuda:0')

In [13]:
molecules.Enuc[2] # TiS. MOPAC Enuc = 176.1518 EV

tensor(176.1491, device='cuda:0')

In [14]:
molecules.Enuc[3] # BrCl. MOPAC Enuc = 391.1161 EV

tensor(391.1087, device='cuda:0')

In [15]:
molecules.Enuc[4] # CrTi. MOPAC Enuc = 251.0941 EV  !!!!!!!!!!!!!!!!!!!!!

tensor(145.6016, device='cuda:0')